In [ ]:
from googleapiclient.discovery import build

import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import pickle
from IPython.display import display
import sys


In [ ]:
month_mapper = {'jan':'01','feb':'02','mar':'03','apr':'04','may':'05','jun':'06','jul':'07','aug':'08','sep':'09','oct':'10','nov':'11','dec':'12','january':'01','february':'02','march':'03','april':'04','june':'06','july':'07','august':'08','sept':'09','september':'09','october':'10','november':'11','december':'12'}
#print(month_mapper)
category_mapper = {0: 'religious', 1:'graphics', 2: 'technology',3:'technology',4:'technology', 5:'technology',6:'forsale',7:'automobile',8:'automobile',9:'sport',10:'sport',11:'science',12:'science',13:'science/entertainment',14:'space',15:'religious',16:'politics.guns',
    17:'politics',18:'politics',19:'religious'}


In [ ]:
def customsearchengine(name):
    api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" #google api key
    resource = build("customsearch", 'v1', developerKey=api_key).cse()
    result = resource.list(q=name, cx='xxxxxxxxxxx').execute() #custom search service key
    #print(result)
    return result
def getdict(result):
    
    temp_dict = dict()

    #snippet
    #i=0
    for item in result['items']:
        temp_dict[item['link']] = None
    return temp_dict

def webscrapping(temp_dict,result):

    date_lst=[]
    #page =2
    #url of the page that we want to Scarpe
    #+str() is used to convert int datatype of the page no. and concatenate that to a URL for pagination purposes.
    #url = 'https://www.politifact.com/factchecks/list/?page='+str(page)
    #i = "narendra_modi"
    #url = result2['items'][0]['link']
    for item in result['items']:
        val = ''
    #for item in temp_dict.keys():
        url = item['link']
        ap_date = ''
        if re.match(r'.*\.cnbc\.com',url):
            print("url:",url)
            html_content = requests.get(url).text

            # Parse the html content
            soup = BeautifulSoup(html_content, "lxml")
            content =  soup.find('div',{'class':'ArticleBody-articleBody'})
            #print(content)
            if content is None:
                
                content =  soup.find('div',{'class':'PageBuilder-styles-makeit-col-9--4VOfc PageBuilder-styles-makeit-col--1vq1j PageBuilder-styles-makeit-article--1sqIm'})
                if content is None:
                    del temp_dict[item['link']]
                    continue
            #val= ''
            for i in content.findAll('div'):
                val =val+ '' + i.text

                val = re.sub('[^A-Za-z0-9]+', ' ', val)
            val = ''.join([i for i in val if not i.isdigit()])
            if val == '':
                del temp_dict[item['link']]

                continue

            date =  soup.find('time')
            if date is None:
                ap_date = 'Not able to extarct the date'
                date_lst.append(ap_date)
            else:
                temp = re.findall(r'.+?Published\s.+?\,\s(\w+)\s(\d+)\s(\d+)',str(date))
                if temp is not None:
                    month = month_mapper[temp[0][0].lower()]
                    #print(month)
                    ap_date = temp[0][1]+"-"+month+"-"+temp[0][2]
                    #print(ap_date)
                    date_lst.append(ap_date)
                else:
                    ap_date = 'Not able to extarct the date'
                    date_lst.append(ap_date)
                
                #print(temp)

                

            temp_dict[item['link']] = val
        elif re.match(r'.*\.usatoday\.com',url):
            print("url:",url)
            html_content = requests.get(url).text

            # Parse the html content
            soup = BeautifulSoup(html_content, "lxml")
            content =  soup.find('main',{'class':'gnt_cw'})
            if content is None:
                content =  soup.find('div',{'class':'_3YYSt clearfix'})
                #print(content)
                if content is None:
                    content =  soup.find('div',{'class':'article-inner theme-light'})
                    if content is None:
                        del temp_dict[item['link']]
                        continue
                    else:
                        #val= ''
                        for i in content.findAll('p'):
                            val =val+ '' + i.text
                            val = re.sub('[^A-Za-z0-9]+', ' ', val)
                        #print(item['link'])
                        #print(val)


            
            if len(val) == 0:
                for i in content.findAll('div'):
                    val =val+ '' + i.text

                    val = re.sub('[^A-Za-z0-9]+', ' ', val)


            val = ''.join([i for i in val if not i.isdigit()])
            if len(val) == 0:
                del temp_dict[item['link']]
                continue

            date =  soup.find('div',{'class':'gnt_ar_dt'})
            if date is None:
                ap_date = 'Not able to extarct the date'
                date_lst.append(ap_date)

            else:
                temp = re.findall(r'.+?Updated\:.+?\sET\s(\w+)[.]\s(\d+)\,\s(\d+)',str(date))
                if len(temp) == 0:
                    temp = re.findall(r'.+?Updated\:.+?\sET\s(\w+)\s(\d+)\,\s(\d+)',str(date))
                    if len(temp)== 0:
                        temp = re.findall(r'.+?Published\s.+?ET\s(\w+)[. ]\s(\d+)\,\s(\d+)',str(date))
                        if len(temp)== 0:
                            temp = re.findall(r'.+?Published\s.+?\sET\s(\w+)\s(\d+)\,\s(\d+)',str(date))
                            if len(temp) == 0:

                                #date_lst.append("couldn't extarct date")
                                #continue
                                if not date:
                                    date =  soup.find('span',{'class':'updated'})
                                    #print(date)
                                    temp = re.findall(r'.+?(\d+)\-(\d+)\-(\d+)',str(date))
                                    #print(temp)
                                    #ap_date = temp[0][1]+"-"+temp[0][0]+"-"+temp[0][2]
                if len(temp)>0 :
                    if temp[0][0].lower() in month_mapper.keys():


                        month = month_mapper[temp[0][0].lower()]
                        #print(month)
                        if len(temp[0][1]) ==1:
                            ap_date = "0"+temp[0][1]+"-"+month+"-"+temp[0][2]
                        else:
                            ap_date = temp[0][1]+"-"+month+"-"+temp[0][2]
                        #print(ap_date)
                    else:
                        ap_date = temp[0][1]+"-"+temp[0][0]+"-"+temp[0][2]
                        #print(ap_date)
                date_lst.append(ap_date)
                    #temp_dict[item] = val
            #print(item['link'])
            #print(val)
            temp_dict[item['link']] = val
        elif re.match(r'.*\.politico\.com',url):
            print("url:",url)
            html_content = requests.get(url).text

            # Parse the html content
            soup = BeautifulSoup(html_content, "lxml")
            content =  soup.find('div',{'class':'page-content page-content--story story-type--core'})
            if content is None:
                content =  soup.find('div',{'class':'story-text'})
                if content is None:
                    del temp_dict[item['link']]
                    continue

            #val= ''
            for i in content.findAll('div'):
                val =val+ '' + i.text

                val = re.sub('[^A-Za-z0-9]+', ' ', val)
            val = ''.join([i for i in val if not i.isdigit()])
            if val == '':
                del temp_dict[item['link']]

                continue
            date =  soup.find('p',{'class':'story-meta__updated'})
            if date is not None:
                temp = re.findall(r'.+?Updated.+?(\d+)\-(\d+)-(\d+)',str(date))
                if temp is not None:
                    ap_date = temp[0][2]+"-"+temp[0][1]+"-"+temp[0][0]
                    date_lst.append(ap_date)
                else:
                    ap_date = 'Not able to extarct the date'
                    date_lst.append(ap_date)

            else:
                date =  soup.find('p',{'class':'story-meta__timestamp'})
                #print(date)
                if date is not None:
                    temp = re.findall(r'.+?(\d+)\-(\d+)\-(\d+).+?',str(date))
                    if temp is not None:
                        ap_date = temp[0][1]+"-"+temp[0][0]+"-"+temp[0][2]
                        date_lst.append(ap_date)
                    else:
                        ap_date = 'Not able to extarct the date'
                        date_lst.append(ap_date)
                else:
                    date =  soup.find('time')
                    #print()
                    #temp = re.findall(r'.+?\>(\d+)\/(\d+)\/(\d+)',str(date))
                    temp = re.findall(r'.+?(\d+)\-(\d+)\-(\d+).+?',str(date))
                    #print(temp)
                    if temp is not None:
                        ap_date = temp[0][1]+"-"+temp[0][0]+"-"+temp[0][2]
                        date_lst.append(ap_date)
                    else:
                        ap_date = 'Not able to extarct the date'
                        date_lst.append(ap_date)
            temp_dict[item['link']] = val
        elif re.match(r'.*\.bbc\.com',url):
            print("url:",url)
            html_content = requests.get(url).text

            # Parse the html content
            soup = BeautifulSoup(html_content, "lxml")
            content =  soup.find('div',{'class':'ssrcss-181c4hk-SectionWrapper eustbbg3'})
            if content is None:
                del temp_dict[item['link']]
                continue
            #val= ''
            for i in content.findAll('div'):
                val =val+ '' + i.text

                val = re.sub('[^A-Za-z0-9]+', ' ', val)
            val = ''.join([i for i in val if not i.isdigit()])
            if val == '':
                del temp_dict[item['link']]

                continue

            date =  soup.find('time')
            if date is None:
                ap_date = 'Not able to extarct the date'
                date_lst.append(ap_date)
            else:
                temp = re.findall(r'datetime\=\"([0-9]+)\-([0-9]+)\-([0-9]+).*',str(date))
                if temp is not None:
                    ap_date = temp[0][2]+"-"+temp[0][1]+"-"+temp[0][0]
                    date_lst.append(ap_date)
                else:
                    ap_date = 'Not able to extarct the date'
                    date_lst.append(ap_date)
                #ap_date = temp[0][2]+"-"+temp[0][1]+"-"+temp[0][0]
                #date_lst.append(ap_date)
            temp_dict[item['link']] = val
        elif re.match(r'.*\.cnn\.com',url):
            print("url:",url)
            html_content = requests.get(url).text

            # Parse the html content
            soup = BeautifulSoup(html_content, "lxml")
            content =  soup.find('div',{'class':'l-container'})
            #print(content)
            if content is None:
                del temp_dict[item['link']]
                continue

            #val= ''
            for i in content.findAll('div'):
                val =val+ '' + i.text

                val = re.sub('[^A-Za-z0-9]+', ' ', val)
            val = ''.join([i for i in val if not i.isdigit()])
            if val == '':
                del temp_dict[item['link']]

                continue

            date =  soup.find('p',{'class':'update-time'})
            if date is None:
                ap_date = 'Not able to extarct the date'
                date_lst.append(ap_date)
            else:
                #print(date)
                temp = re.findall(r'\s(\w+)\s([0-9]+),\s([0-9]+).*',str(date))
                if temp is not None:
                    month = month_mapper[temp[0][0].lower()]
                    #print(month)
                    ap_date = temp[0][1]+"-"+month+"-"+temp[0][2]
                    ap_date
                    date_lst.append(ap_date)
                else:
                    ap_date = 'Not able to extarct the date'
                    date_lst.append(ap_date)
                #print(temp)
                

            temp_dict[item['link']] = val

    return temp_dict, date_lst
    #temp_dict
def getdataframe(temp_dict,date_lst):
    idx = ['Links','Data']
    rona = pd.DataFrame.from_dict(temp_dict, orient = 'index')
    rona = rona.reset_index()
    rona= rona.rename(columns = {'index':'Links',0:'Data'})
    #rona['Date Published'] = date_lst
    return rona

def getmodel():
    #to import the saved model
    with open(r".\svm_v1.pkl", 'rb') as file:  
        text_clf_svm = pickle.load(file)
    return text_clf_svm

def getfinaldf(rona):
    text_clf_svm = getmodel()
    predict = text_clf_svm.predict(rona['Data'].to_list())
    rona['Category'] = predict
    rona['Category'] = rona['Category'].apply(lambda x: category_mapper[x])
    return rona

def getfinalresult(rona):
    catper=[]
    totallen = len(rona.index)
    tempStore = rona['Category'].unique()
    #print(tempStore)
    for i in tempStore:
        temp_df = rona[rona['Category'] == i]
        totalpol = len(temp_df)
        perpol = int(totalpol/totallen*100)
        #print(perpol)
        catper.append(perpol)
    return tempStore,catper

def getdata(name):
    result = customsearchengine(name)
    #print(result)
    if result['searchInformation']['totalResults'] == '0':
        #print("Unable to fetch any news articles related to "+name)
        sys.exit("Unable to fetch any news articles related to "+name)
        #exit
    temp_dict = getdict(result)
    temp_dict,date_lst = webscrapping(temp_dict,result)
    rona =getdataframe(temp_dict,date_lst)
    final_df = getfinaldf(rona)
    display(rona)
    tempStore,catper = getfinalresult(rona)
    
    print("Category Exposure:")
    for i in range(len(tempStore)):
        print(str(i+1)+'.'+tempStore[i]+':'+str(catper[i])+'%')


In [ ]:
name = input("Enter the applicant's name:")
getdata(name)